In [ ]:
#อนุญาตการเข้าถึง Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install flask pyngrok


In [ ]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok
import joblib
import numpy as np
from PIL import Image
import base64
import io

app = Flask(__name__)

# โหลดโมเดลที่บันทึกเป็นไฟล์ joblib
model = joblib.load('/content/drive/MyDrive/ฺButterfly/best_random_forest_model.joblib')

# สายพันธุ์ที่โมเดลสามารถจำแนกได้
classes = ['ADONIS', 'AFRICAN GIANT SWALLOWTAIL', 'AMERICAN SNOOT', 'AN 88', 'APPOLLO',
           'ATALA', 'BANDED ORANGE HELICONIAN', 'BANDED PEACOCK', 'BECKERS WHITE',
           'BLACK HAIRSTREAK', 'BLUE MORPHO', 'BLUE SPOTTED CROW', 'BROWN SIPROETA',
           'CABBAGE WHITE', 'CAIRNS BIRDWING', 'CHECQUERED SKIPPER', 'CHESTNUT',
           'CLEOPATRA', 'CLODIUS PARNASSIAN', 'CLOUDED SULPHUR', 'COMMON BANDED AWL',
           'COMMON WOOD-NYMPH', 'COPPER TAIL', 'CRECENT', 'CRIMSON PATCH',
           'DANAID EGGFLY', 'EASTERN COMA', 'EASTERN DAPPLE WHITE',
           'EASTERN PINE ELFIN', 'ELBOWED PIERROT', 'GOLD BANDED', 'GREAT EGGFLY',
           'GREAT JAY', 'GREEN CELLED CATTLEHEART', 'GREY HAIRSTREAK', 'INDRA SWALLOW',
           'IPHICLUS SISTER', 'JULIA', 'LARGE MARBLE', 'MALACHITE', 'MANGROVE SKIPPER',
           'MESTRA', 'METALMARK', 'MILBERTS TORTOISESHELL', 'MONARCH', 'MOURNING CLOAK',
           'ORANGE OAKLEAF', 'ORANGE TIP', 'ORCHARD SWALLOW', 'PAINTED LADY',
           'PAPER KITE', 'PEACOCK', 'PINE WHITE', 'PIPEVINE SWALLOW', 'POPINJAY',
           'PURPLE HAIRSTREAK', 'PURPLISH COPPER', 'QUESTION MARK', 'RED ADMIRAL',
           'RED CRACKER', 'RED POSTMAN', 'RED SPOTTED PURPLE', 'SCARCE SWALLOW',
           'SILVER SPOT SKIPPER', 'SLEEPY ORANGE', 'SOOTYWING', 'SOUTHERN DOGFACE',
           'STRAITED QUEEN', 'TROPICAL LEAFWING', 'TWO BARRED FLASHER', 'ULYSES',
           'VICEROY', 'WOOD SATYR', 'YELLOW SWALLOW TAIL', 'ZEBRA LONG WING']

# HTML template
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="th">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>ระบบจำแนกสายพันธุ์ผีเสื้อ</title>
    <style>
        body {
            font-family: 'Kanit', sans-serif;
            background-color: #f0f0f0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
        }
        .container {
            background-color: white;
            padding: 2rem;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            text-align: center;
        }
        h1 {
            color: #4a4a4a;
        }
        form {
            margin-top: 1rem;
        }
        input[type="file"] {
            display: none;
        }
        .custom-file-upload {
            border: 1px solid #ccc;
            display: inline-block;
            padding: 6px 12px;
            cursor: pointer;
            background-color: #f8f9fa;
            color: #4a4a4a;
            border-radius: 4px;
            transition: all 0.3s;
        }
        .custom-file-upload:hover {
            background-color: #e9ecef;
        }
        input[type="submit"] {
            background-color: #007bff;
            color: white;
            border: none;
            padding: 10px 20px;
            margin-top: 1rem;
            cursor: pointer;
            border-radius: 4px;
            transition: background-color 0.3s;
        }
        input[type="submit"]:hover {
            background-color: #0056b3;
        }
        .result {
            margin-top: 2rem;
            font-size: 1.2rem;
            color: #28a745;
        }
        #preview {
            max-width: 300px;
            max-height: 300px;
            margin-top: 1rem;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>ระบบจำแนกสายพันธุ์ผีเสื้อ</h1>
        <form action="/predict" method="post" enctype="multipart/form-data">
            <label for="file-upload" class="custom-file-upload">
                เลือกรูปภาพ
            </label>
            <input id="file-upload" type="file" name="file" accept="image/*" onchange="previewImage(event)">
            <br>
            <img id="preview" src="" alt="Preview" style="display:none;">
            <br>
            <input type="submit" value="วิเคราะห์">
        </form>
        {% if result %}
        <div class="result">
            <h2>ผลการวิเคราะห์</h2>
            <p>สายพันธุ์ที่ทำนายได้: {{ result }}</p>
            {% if image_data %}
            <img src="data:image/jpeg;base64,{{ image_data }}" alt="Uploaded Image" style="max-width: 300px; max-height: 300px;">
            {% endif %}
        </div>
        {% endif %}
    </div>
    <script>
        function previewImage(event) {
            var reader = new FileReader();
            reader.onload = function(){
                var output = document.getElementById('preview');
                output.src = reader.result;
                output.style.display = 'block';
            };
            reader.readAsDataURL(event.target.files[0]);
        }
    </script>
</body>
</html>
'''

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['file']
    if not file:
        return 'กรุณาอัปโหลดไฟล์รูปภาพ'

    # โหลดรูปภาพและเตรียมข้อมูล
    img = Image.open(file)
    img_display = img.copy()
    img = img.resize((128, 128))  # ปรับขนาดให้เข้ากับโมเดล
    img = np.array(img) / 255.0  # ปรับค่าให้อยู่ในช่วง 0-1
    img = img.flatten().reshape(1, -1)  # แปลงเป็นเวกเตอร์แล้วแปลงขนาดสำหรับโมเดล

    # ทำการทำนาย
    prediction = model.predict(img)
    predicted_class = classes[int(prediction[0])]

    # แปลงรูปภาพเป็น base64 เพื่อแสดงผล
    buffered = io.BytesIO()
    img_display.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode()

    return render_template_string(HTML_TEMPLATE, result=predicted_class, image_data=img_str)

# ตั้งค่า ngrok authtoken
ngrok.set_auth_token("2mqEaqKTNmNXfdPYThWpo9wbhY5_hmgkuX3hbm3NamfvSKsK")

# เปิดใช้งาน ngrok
public_url = ngrok.connect(5000)
print(f"Flask app is running at {public_url}")

# รัน Flask app
if __name__ == '__main__':
    app.run()

Flask app is running at NgrokTunnel: "https://d0a7-35-196-184-18.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Oct/2024 14:43:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2024 14:43:57] "GET /favicon.ico HTTP/1.1" 404 -
